In [2]:
import os
import random
import shutil
import yaml
from typing import List, Tuple


In [3]:
def clear_directory(directory: str) -> None:
    """
    Leert den angegebenen Ordner, indem alle Dateien und Unterordner gelöscht werden.
    
    Args:
        directory: Zu leerender Ordner
    """
    if os.path.exists(directory):
        for item in os.listdir(directory):
            item_path = os.path.join(directory, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
        print(f"Ordner geleert: {directory}")
    else:
        print(f"Ordner existiert nicht: {directory}")

In [4]:
def read_classes(classes_file: str) -> List[str]:
    """
    Liest Klassennamen aus einer Textdatei.
    
    Args:
        classes_file: Pfad zur Textdatei mit Klassennamen
        
    Returns:
        Liste der Klassennamen
    """
    if not os.path.exists(classes_file):
        print(f"Warnung: Klassendatei {classes_file} nicht gefunden.")
        return ['item']
    
    with open(classes_file, 'r') as f:
        classes = [line.strip() for line in f.readlines() if line.strip()]
    
    return classes

In [5]:
def organize_dataset(
    src_images_dir: str,
    src_labels_dir: str,
    dst_root_dir: str,
    split_ratio: Tuple[float, float, float] = (0.7, 0.2, 0.1),
    classes_file: str = None,
    clear_existing: bool = False
) -> None:
    """
    Organisiert Bild- und Label-Dateien und erstellt data.yaml für YOLOv8.
    
    Args:
        src_images_dir: Quellordner mit .jpg Bildern
        src_labels_dir: Quellordner mit .txt Labels
        dst_root_dir: Zielordner für die neue Struktur
        split_ratio: Verhältnisse für (Train, Test, Valid)-Split
        classes_file: Pfad zur Textdatei mit Klassennamen
        clear_existing: Bei True werden bestehende Dateien im Zielordner gelöscht
    """
    # Prüfe, ob die Summe der Verhältnisse 1 ergibt
    if sum(split_ratio) != 1.0:
        raise ValueError("Die Summe der Split-Verhältnisse muss 1.0 ergeben")
    
    # Lese Klassennamen
    if classes_file is None:
        classes_file = os.path.join(dst_root_dir, 'classes.txt')
    
    class_names = read_classes(classes_file)
    print(f"Gefundene Klassen: {class_names}")
    
    # Bei Bedarf bestehende Daten löschen
    if clear_existing and os.path.exists(dst_root_dir):
        print(f"Lösche bestehende Daten in {dst_root_dir}...")
        
        # Lösche die Ordner für jeden Split
        for split_name in ['train', 'test', 'valid']:
            split_dir = os.path.join(dst_root_dir, split_name)
            if os.path.exists(split_dir):
                shutil.rmtree(split_dir)
                print(f"Ordner gelöscht: {split_dir}")
        
        # Lösche data.yaml wenn vorhanden
        yaml_path = os.path.join(dst_root_dir, 'data.yaml')
        if os.path.exists(yaml_path):
            os.remove(yaml_path)
            print(f"Datei gelöscht: {yaml_path}")
    
    # 1) Erstelle eine Liste von zusammengehörigen Bild/Label-Paaren
    image_files = [f for f in os.listdir(src_images_dir) if f.endswith('.jpg')]
    pairs = []
    
    for img_file in image_files:
        base_name = os.path.splitext(img_file)[0]
        label_file = base_name + '.txt'
        
        if os.path.exists(os.path.join(src_labels_dir, label_file)):
            pairs.append((img_file, label_file))
    
    # 2) Mische die Liste und erstelle den Split
    random.shuffle(pairs)
    
    n_samples = len(pairs)
    n_train = int(n_samples * split_ratio[0])
    n_test = int(n_samples * split_ratio[1])
    
    train_pairs = pairs[:n_train]
    test_pairs = pairs[n_train:n_train + n_test]
    valid_pairs = pairs[n_train + n_test:]
    
    # 3) Erstelle die Zielordnerstruktur und verschiebe die Dateien
    splits = {
        'train': train_pairs,
        'test': test_pairs,
        'valid': valid_pairs
    }
    
    for split_name, file_pairs in splits.items():
        # Erstelle Zielordner
        img_dir = os.path.join(dst_root_dir, split_name, 'images')
        label_dir = os.path.join(dst_root_dir, split_name, 'labels')
        
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(label_dir, exist_ok=True)
        
        # Verschiebe Dateien
        for img_file, label_file in file_pairs:
            src_img_path = os.path.join(src_images_dir, img_file)
            src_label_path = os.path.join(src_labels_dir, label_file)
            
            dst_img_path = os.path.join(img_dir, img_file)
            dst_label_path = os.path.join(label_dir, label_file)
            
            shutil.copy2(src_img_path, dst_img_path)
            shutil.copy2(src_label_path, dst_label_path)
    
    # 4) Erstelle data.yaml für YOLOv8
    abs_path = os.path.abspath(dst_root_dir)
    data_yaml = {
        'path': abs_path,
        'train': os.path.join(abs_path, 'train', 'images'),
        'val': os.path.join(abs_path, 'valid', 'images'),
        'test': os.path.join(abs_path, 'test', 'images'),
        'nc': len(class_names),
        'names': class_names
    }
    
    # Schreibe YAML-Datei
    yaml_path = os.path.join(dst_root_dir, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(data_yaml, f, sort_keys=False)
    
    print(f"Datensatz aufgeteilt: {len(train_pairs)} Train, {len(test_pairs)} Test, {len(valid_pairs)} Valid")
    print(f"data.yaml erstellt: {yaml_path}")

# Beispielaufruf:
# organize_dataset(
#     src_images_dir="images",
#     src_labels_dir="labels",
#     dst_root_dir="fb_training_images",
#     split_ratio=(0.7, 0.2, 0.1),
#     classes_file="fb_training_images/classes.txt",
#     clear_existing=True
# )

In [6]:
organize_dataset(
     src_images_dir="images",
     src_labels_dir="labels",
     dst_root_dir="fb_training_images",
     split_ratio=(0.7, 0.2, 0.1),
     classes_file="fb_training_images/classes.txt",
     clear_existing=True  # Auf True setzen, um bestehende Dateien zu löschen
)

Gefundene Klassen: ['ball', 'cornerpoints', 'goal', 'goalkeeper', 'period', 'player', 'ref', 'scoreboard', 'time']
Lösche bestehende Daten in fb_training_images...
Ordner gelöscht: fb_training_images/train
Ordner gelöscht: fb_training_images/test
Ordner gelöscht: fb_training_images/valid
Datei gelöscht: fb_training_images/data.yaml
Datensatz aufgeteilt: 86 Train, 24 Test, 13 Valid
data.yaml erstellt: fb_training_images/data.yaml
